In [ ]:
from scipy.integrate import odeint
import math 
import numpy as np
import astroabc
import GPy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

%matplotlib inline


In [ ]:
def model_deriv(y, t, param):
    #Define parameters
    a,b,c=param

    #define states
    X,Y,Z=y


    #define derivatives
    dX_dt = -Y - Z
    dY_dt = X + a*Y
    dZ_dt = b + Z*(X-c)
    
    return dX_dt,dY_dt,dZ_dt
    
def model_sol(param):
    
    y0 = [4.,4.,0.1]
    time = np.linspace(0, 500, 1000)

    solution = odeint(model_deriv, y0, time, args=(param,))
    return np.array(solution)

In [ ]:
time = np.linspace(0, 500, 1000)

sol=model_sol([0.1,0.1,14.])
Y=np.ones((len(time),3))
for states in range(3):
    Y[:,states] = sol[:,states] + np.random.randn(len(time),)*(0.1*np.mean(sol[:,states]))

plt.plot(Y[:,0],Y[:,1])
plt.plot(sol[:,0],sol[:,1])

In [ ]:
kernX = GPy.kern.Matern32(1) + GPy.kern.Bias(1)
kernY = GPy.kern.Matern32(1) + GPy.kern.Bias(1)
kernZ = GPy.kern.Matern32(1) + GPy.kern.Bias(1)
input_time=time.reshape(1000,1)#[-500:]
modelX = GPy.models.GPRegression(input_time, Y[:,0].reshape(len(input_time),1), kernX)#[-500:,0]
modelY = GPy.models.GPRegression(input_time, Y[:,1].reshape(len(input_time),1), kernX)
modelZ = GPy.models.GPRegression(input_time, Y[:,2].reshape(len(input_time),1), kernX)
#modelZ.Gaussian_noise.variance=0.23

In [ ]:
modelX.optimize_restarts(optimizer='lbfgs',messages=False,num_restarts = 1)
modelY.optimize_restarts(optimizer='lbfgs',messages=False,num_restarts = 1)
modelZ.optimize_restarts(optimizer='lbfgs',messages=False,num_restarts = 1)

In [4]:
def smooth_gp(traces, input_time, gp_kernel, true_sol=None, optim_restarts = 1):
    time_len = len(input_time)
    gp_models = []
    state = []
    velocity = []
    for states in xrange(traces.shape[1]):
        kern = GPy.kern.Matern32(1) + GPy.kern.Bias(1) 
        gp_models.append(GPy.models.GPRegression(input_time, 
                                    traces[-time_len:,states].reshape(len(input_time),1), kern))
    #for states in xrange(traces.shape[1]):
    #    gp_models[states].optimize_restarts(optimizer='lbfgs',messages=False,
    #                                        num_restarts = optim_restarts)
    start = 400
    stop = 500
    window = [start,stop]
    for states in xrange(traces.shape[1]):
        gp_models[states].optimize_restarts(optimizer='lbfgs',messages=False,
                                           num_restarts = optim_restarts)
        state.append(gp_models[states].posterior_samples_f(input_time,size=10))
        velocity.append(gp_models[states].predictive_gradients(input_time)
                       [0].reshape(len(input_time),))
        gp_models[states].plot_noiseless(window)
        if not(true_sol) == None:
            plt.plot(input_time[-500:],true_sol[-500:,i],
                     color='#ff7f0e',lw=1.5,label='state '+str(i+1))
        plt.legend()
        plt.xlabel('Time')
        plt.ylabel('Value')
    return state, velocity, gp_models

In [5]:
input_time=time.reshape(1000,1)
gp_kernel = GPy.kern.Matern32(1) + GPy.kern.Bias(1)
xbar, dxbar, gps = smooth_gp(Y, input_time, gp_kernel)

NameError: name 'time' is not defined

In [ ]:
def simulation_rhs_f(trial):
    stateLen = 1000
    sample = 10
    xbar_X, xbar_Y, xbar_Z = xbar
    rhsf_X = np.ones((stateLen,sample))
    rhsf_Y = np.ones((stateLen,sample))
    rhsf_Z = np.ones((stateLen,sample))
    for t in range(stateLen):
        for j in range(sample):
            rhsf_X[t,j] = -xbar_Y[t,j] - xbar_Z[t,j]
            rhsf_Y[t,j] = xbar_X[t,j] + trial[0]*xbar_Y[t,j]
            rhsf_Z[t,j] = trial[1] + xbar_Z[t,j]*(xbar_X[t,j]-trial[2])
    
    if np.any(np.array([trial])<0.0):
        rhsf_X[0,:]=-1000.
        rhsf_Y[0,:]=-1000.
        rhsf_Z[0,:]=-1000.
       
    return [rhsf_X.mean(axis=1),rhsf_Y.mean(axis=1),rhsf_Z.mean(axis=1)]

In [ ]:
def dist_metric(d,x):
    dist=0.0
    for states in range(3):
        if np.all(np.array(x[states][0]==-1000.0)):
            dist += np.inf
        else:
            dist += np.sum((d[states]-x[states])**2)    
            
    return dist  

In [ ]:
tt=simulation_rhs_f([0.1,0.1,14])
plt.figure(figsize=(15, 7.5))
plt.plot(input_time,dxbar[2])
plt.plot(input_time,tt[2])
np.sum((dxbar[2]-tt[2])**2)

In [ ]:
dist=dist_metric(dxbar,tt)
trial_dist=dist*3

In [ ]:
data = dxbar
#priors =  [('uniform', [0.0,2.]), ('uniform', [0.0,2.]), ('uniform', [0.0,30.0])]
priors =  [('gamma', [2.0,1.]), ('uniform', [0.0,2.]), ('uniform', [5.0,40.0])]

In [ ]:
prop={'dfunc':dist_metric, 'outfile':"rossler.txt", 'verbose':1, 'adapt_t': True,\
     'tol_type':"linear" ,'pert_kernel': 2, 'variance_method': 0}

In [ ]:
#'threshold': 0.7,

In [ ]:
sampler = astroabc.ABC_class(3,100,data,[trial_dist,100],30,priors,**prop)

In [ ]:
sampler.sample(simulation_rhs_f)

In [ ]:
def model_sol_pos(param,init):
    y0 = init
    time = np.linspace(0, 500, 500000)
    solution = odeint(model_deriv, y0, time, args=(param,))
    return solution

In [ ]:
init=[xbar[0][0,:].mean(axis=0),xbar[1][0,:].mean(axis=0),xbar[2][0,:].mean(axis=0)]
init

In [ ]:
def plot_pos(soln_fn, params, sampler):
    center = []
    upper = []
    lower = []
    for par in xrange(sampler.nparams):
        center.append(np.mean(sampler.theta[step][:,par]))
        upper.append(np.mean(sampler.theta[step][:,par])+1.96*np.std(sampler.theta[step][:,par]))
        lower.append(np.mean(sampler.theta[step][:,par])+1.96*np.std(sampler.theta[step][:,par]))

In [ ]:
step=29
center = [np.mean(sampler.theta[step][:,0]),\
                     np.mean(sampler.theta[step][:,1]),\
                     np.mean(sampler.theta[step][:,2])]
upper = [np.mean(sampler.theta[step][:,0])+2*np.std(sampler.theta[step][:,0])/10,\
                     np.mean(sampler.theta[step][:,1])+2*np.std(sampler.theta[step][:,1])/10,\
                     np.mean(sampler.theta[step][:,2])+2*np.std(sampler.theta[step][:,2])/10]
lower = [np.mean(sampler.theta[step][:,0])-2*np.std(sampler.theta[step][:,0])/10,\
                     np.mean(sampler.theta[step][:,1])-2*np.std(sampler.theta[step][:,1])/10,\
                     np.mean(sampler.theta[step][:,2])-2*np.std(sampler.theta[step][:,2])/10]

#print(upper)7.26437805, -3.24197867, 35.79321362
mean_soln = model_sol_pos(center,init)
lower_soln = model_sol_pos(lower,init)
upper_soln = model_sol_pos(upper,init)
true_init = [7.26437805, -3.24197867, 35.79321362]
true_soln = model_sol_pos([0.1,0.1,14],init)

plt.figure(figsize=(15, 7.5))
fig = plt.figure(figsize=(25, 17.5))
ax = fig.add_subplot(121, projection='3d')
plt.plot(true_soln[:,0],true_soln[:,1],true_soln[:,2])
ax = fig.add_subplot(122, projection='3d')
plt.plot(lower_soln[:,0],lower_soln[:,1],lower_soln[:,2])


In [ ]:
plt.figure(figsize=(15, 7.5))
plt.plot(mean_soln[-10000:,2],lw=4)
plt.plot(lower_soln[-10000:,2],'--',color='black')
plt.plot(upper_soln[-10000:,2],'--',color='gray')
plt.plot(true_soln[-10000:,2])

In [ ]:
results=np.loadtxt("rossler.txt",skiprows=1)
results.shape

In [ ]:
par=results[2800:2900,0:4]
time = np.linspace(0, 500, 500000)
conf=2*np.std(par,axis=0)/10
mean=np.mean(par,axis=0)
dmin=np.argsort(par[:,3])
param=par[dmin[:20],:3]
param[0,:]
plt.hist(par[:,2],10)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(25, 17.5))

new_values = []
for ind in range(len(param)):
    
    ppc_sol=model_sol_pos(param[ind],init)
    new_values.append(ppc_sol)
new_values = np.array(new_values)
ax = fig.add_subplot(121, projection='3d')
plt.plot(new_values[0,:,0],new_values[0,:,1],new_values[0,:,2],color='green', alpha=1.5)   
ax = fig.add_subplot(122, projection='3d')
plt.plot(true_soln[:,0],true_soln[:,1],true_soln[:,2], color='black', alpha=1.5)     


plt.figure(figsize=(15, 7.5))
plt.plot(time[-100000:], new_values[0][-100000:,0], color='yellow', alpha=0.1, label='inferred concentration')
for v in new_values[1:]:
    plt.plot(time[-100000:], v[-100000:,0], color='green', alpha=0.1)
plt.plot(time[-100000:], true_soln[-100000:,0], color='black', lw=4, label='mean of inferred')
#plt.plot(time, mean_soln[:,0], color='#ff7f0e', lw=4, label='original concentration')
#plt.plot(time, Y, 'o', color='#7f7f7f', ms=6.5, label='data points')
plt.legend()
plt.xlabel('Time')
plt.ylabel('concentration')

In [ ]:
mean_values = np.mean(new_values, axis=0)
plt.plot(mean_values[390000:500000,2])
plt.plot(true_soln[390000:500000,2])


In [ ]:
new_values = []
for ind in range(len(par)):
    ppc_sol=model_sol_pos(par[ind],init)
    new_values.append(ppc_sol)
new_values = np.array(new_values)
mean_values = np.mean(new_values, axis=0)
new_values.shape


In [ ]:

fig = plt.figure(figsize=(25, 17.5))
ax = fig.add_subplot(121, projection='3d')
plt.plot(true_soln[:,0],true_soln[:,1],true_soln[:,2])
plt.plot(mean_values[:,0],mean_values[:,1],mean_values[:,2])

In [ ]:
plt.plot(true_soln[:,0],true_soln[:,1])

In [ ]:
plt.plot(new_values[36,:,0],new_values[36,:,1])

In [ ]:
mean_soln = model_sol_pos(mean-conf,[4,4,0.1])
plt.plot(mean_soln[:,0],mean_soln[:,1])